In [ ]:
# Minimal FastHTML WebSocket Chat
from typing import List, Dict, Union, Optional, Callable, Awaitable
import inspect
import asyncio
from fasthtml.common import *
from fasthtml.jupyter import render_ft, JupyUvi

render_ft()

app = FastHTML(exts="ws")
rt = app.route

# In-notebook server (adjust port as needed)
PORT = 10979
server = JupyUvi(app, port=PORT, log_level="info")
A("visit site", href=f"http://localhost:{PORT}")

INFO:     Started server process [9448]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:10979 (Press CTRL+C to quit)
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:10979 (Press CTRL+C to quit)


<div>
<a href="http://localhost:10979">visit site</a><script>if (window.htmx) htmx.process(document.body)</script></div>


INFO:     127.0.0.1:51896 - "WebSocket /ws" 403
INFO:     connection rejected (403 Forbidden)
INFO:     connection rejected (403 Forbidden)
INFO:     connection closed
INFO:     connection closed
INFO:     127.0.0.1:51918 - "WebSocket /ws" [accepted]
INFO:     127.0.0.1:51918 - "WebSocket /ws" [accepted]
INFO:     connection open
INFO:     connection open


INFO:     127.0.0.1:51927 - "GET / HTTP/1.1" 200 OK


INFO:     connection closed
INFO:     127.0.0.1:51928 - "WebSocket /ws" [accepted]
INFO:     127.0.0.1:51928 - "WebSocket /ws" [accepted]
INFO:     connection open
INFO:     connection open


In [2]:
# UI primitives and state
from torch_snippets import AD

style_template = """background: {bg}; text-align: {align}; padding: 10px;"""
styles = AD(
    css={
        "user": style_template.format(bg="#EEE6CA", align="right"),
        "assistant": style_template.format(bg="#E5BEB5", align="left"),
    },
    emojis={"user": "🗣️", "assistant": "🐕‍🦺"},
)


def Card(data: dict):
    role = data["role"]
    content = data.get("content", "")
    pending = data.get("pending", False)
    style = styles.css[role]
    emoji = styles.emojis[role]
    if pending:
        spinner = Span(cls="spinner")
        return Div(f"{emoji} {role}: ", spinner, style=style)
    return Div(f"{emoji} {role}: {content}", style=style)


CHAT_DIV_ID = "chat-cards"
messages: List[Dict[str, str]] = []


def render_chat_list():
    return Div(*[Card(m) for m in messages], id=CHAT_DIV_ID, cls="chat-cards")


def mk_inp():
    return Input(id="msg", placeholder="Type a message...", autofocus=True)


@rt("/")
def home():
    return Div(
        render_chat_list(),
        Form(mk_inp(), id="form", ws_send=True),
        hx_ext="ws",
        ws_connect="/ws",
    )


# Connected users (id->send) for broadcast
users: Dict[str, Callable] = {}


def on_conn(ws, send):
    users[str(id(ws))] = send


def on_disconn(ws):
    users.pop(str(id(ws)), None)


# Default responder (can be replaced)
async def default_responder(text: str) -> str:
    # simulate latency to show spinner
    await asyncio.sleep(2)
    return f"echo: {text}"


responder: Optional[Callable[[str], Union[str, Awaitable[str]]]] = default_responder


@app.ws("/ws", conn=on_conn, disconn=on_disconn)
async def ws(msg: str, send):
    # 1) Append user's message and broadcast immediately
    messages.append({"role": "user", "content": msg})
    for u in list(users.values()):
        try:
            await u(render_chat_list())
        except Exception:
            pass

    # 2) Insert assistant pending placeholder and broadcast
    ph_id = f"ph-{len(messages)}"
    messages.append({"role": "assistant", "content": "", "pending": True, "id": ph_id})
    for u in list(users.values()):
        try:
            await u(render_chat_list())
        except Exception:
            pass

    # 3) Compute reply (sync/async), replace placeholder content, clear pending, broadcast again
    reply_text = ""
    if responder:
        try:
            res = responder(msg)
            reply_text = await res if inspect.isawaitable(res) else res
        except Exception as e:
            reply_text = f"Responder error: {e}"
    # replace pending
    for m in messages:
        if m.get("id") == ph_id:
            m.pop("pending", None)
            m["content"] = str(reply_text)
            break

    for u in list(users.values()):
        try:
            await u(render_chat_list())
        except Exception:
            pass

    # 4) Clear input for sender
    await send(mk_inp())